# 1. Data Preprocessing

## 1.1 Import modules

In [1]:
%pylab inline
from time import time
import numpy as np
from numpy.random import seed
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D, Conv1D
from keras.optimizers import SGD, Adam, Adadelta
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from tensorflow import set_random_seed

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


## 1.2 Set seeds

In [2]:
seed(3141593)
set_random_seed(3141593)

## 1.3 Import train and test data

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## 1.4 Encoding data

In [4]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

In [5]:
train.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [6]:
train.shape

(990, 192)

## 1.5 Standardize train and test data

In [7]:
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train.values)
scaled_test = scaler.transform(test.values)

In [8]:
train.values

array([[0.007812, 0.023438, 0.023438, ..., 0.004883, 0.      , 0.025391],
       [0.005859, 0.      , 0.03125 , ..., 0.000977, 0.039062, 0.022461],
       [0.005859, 0.009766, 0.019531, ..., 0.      , 0.020508, 0.00293 ],
       ...,
       [0.001953, 0.003906, 0.      , ..., 0.027344, 0.      , 0.001953],
       [0.      , 0.      , 0.046875, ..., 0.      , 0.001953, 0.00293 ],
       [0.023438, 0.019531, 0.03125 , ..., 0.023438, 0.025391, 0.022461]])

In [9]:
scaled_train

array([[-0.48661074, -0.13135701, -0.33095592, ..., -0.39487147,
        -0.65214311,  0.26239707],
       [-0.58560191, -0.73488047, -0.02856115, ..., -0.49497401,
         2.18166976,  0.13364087],
       [-0.58560191, -0.48340807, -0.48219201, ..., -0.52001246,
         0.83564129, -0.72463124],
       ...,
       [-0.78358426, -0.63430181, -1.23821765, ...,  0.1807566 ,
        -0.65214311, -0.76756462],
       [-0.88257543, -0.73488047,  0.5762671 , ..., -0.52001246,
        -0.51045972, -0.72463124],
       [ 0.30542003, -0.23196142, -0.02856115, ...,  0.08065405,
         1.18988604,  0.13364087]])

## 1.6 Train-validation split
Use stratified sampling to ensure every class is represented equally.

In [10]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=123)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

## 1.7 Reshape data

In [11]:
feature_number = 64
class_number = len(classes)

# Reshape train data
X_train_r = np.zeros((len(X_train), feature_number, 3))
X_train_r[:, :, 0] = X_train[:, :feature_number]
X_train_r[:, :, 1] = X_train[:, feature_number:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# Reshape validation data
X_valid_r = np.zeros((len(X_valid), feature_number, 3))
X_valid_r[:, :, 0] = X_valid[:, :feature_number]
X_valid_r[:, :, 1] = X_valid[:, feature_number:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

# Reshape test data
test_r = np.zeros((len(scaled_test), feature_number, 3))
test_r[:, :, 0] = scaled_test[:, :feature_number]
test_r[:, :, 1] = scaled_test[:, feature_number:128]
test_r[:, :, 2] = scaled_test[:, 128:]

## 1.8 Data formatting

In [12]:
# Convert to a binary matrix representation of the input.
y_train = np_utils.to_categorical(y_train, class_number)
y_valid = np_utils.to_categorical(y_valid, class_number)
y_cat = np_utils.to_categorical(labels)

# 2. Models

## 2.1 1-Dimensional Neural Network Model

### 2.1.1 1-D CNN Model Architecture

In [13]:
model_cnn = Sequential()
model_cnn.add(Convolution1D(nb_filter=1024, filter_length=6, input_shape=(feature_number, 3)))
model_cnn.add(Activation('relu'))
model_cnn.add(Flatten())
model_cnn.add(Dropout(rate = 1 - 0.2))
model_cnn.add(Dense(2048, activation='relu'))
model_cnn.add(Dense(1024, activation='relu'))
model_cnn.add(Dense(512, activation='relu'))
model_cnn.add(Dense(class_number))
model_cnn.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\sunsh\AppData\Local\conda\conda\envs\python-CZ4041\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=1024, kernel_size=6)`
  


### 2.1.2 1-D CNN Model Training

In [14]:
model_cnn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4, decay=1e-5), metrics=['accuracy'])

model_cnn.fit(X_train_r, y_train, epochs = 100, validation_data=(X_valid_r, y_valid), batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 891 samples, validate on 99 samples
Epoch 1/100
891/891 [==============================] - 9s 10ms/step - loss: 4.2281 - acc: 0.1167 - val_loss: 3.2019 - val_acc: 0.4444
Epoch 2/100
891/891 [==============================] - 2s 2ms/step - loss: 2.1410 - acc: 0.6420 - val_loss: 1.0658 - val_acc: 0.8889
Epoch 3/100
891/891 [==============================] - 2s 2ms/step - loss: 0.6237 - acc: 0.9001 - val_loss: 0.3784 - val_acc: 0.9394
Epoch 4/100
891/891 [==============================] - 2s 2ms/step - loss: 0.2077 - acc: 0.9719 - val_loss: 0.1803 - val_acc: 0.9798
Epoch 5/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0896 - acc: 0.9910 - val_loss: 0.0894 - val_acc: 1.0000
Epoch 6/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0478 - acc: 0.9899 - val_loss: 0.0756 - val_acc: 0.9899
Epoch 7/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0269 - acc: 0.9989 - val_loss: 

Epoch 59/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0320 - acc: 0.9955 - val_loss: 0.0234 - val_acc: 0.9899
Epoch 60/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0092 - acc: 0.9989 - val_loss: 0.0175 - val_acc: 1.0000
Epoch 61/100
891/891 [==============================] - 2s 2ms/step - loss: 9.7897e-04 - acc: 1.0000 - val_loss: 0.0198 - val_acc: 1.0000
Epoch 62/100
891/891 [==============================] - 2s 2ms/step - loss: 9.3557e-04 - acc: 1.0000 - val_loss: 0.0187 - val_acc: 1.0000
Epoch 63/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0283 - val_acc: 0.9899
Epoch 64/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0192 - acc: 0.9978 - val_loss: 0.0200 - val_acc: 1.0000
Epoch 65/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0172 - acc: 0.9989 - val_loss: 0.0220 - val_acc: 0.9899
Epoch 66/100
891/891 [==============================] 

### 2.1.3 1-D CNN Model Prediction

In [15]:
pred_cnn = model_cnn.predict(test_r)

## 2.2 Multi-Layer Perceptron

### 2.2.1 MLP Model Architecture

In [16]:
model_mlp = Sequential()
model_mlp.add(Dense(512, input_dim=192, init='glorot_normal', activation='relu'))
model_mlp.add(Dropout(0.3))
model_mlp.add(Dense(256, activation='sigmoid')) 
model_mlp.add(Dropout(0.3)) 
model_mlp.add(Dense(99, activation='softmax'))

C:\Users\sunsh\AppData\Local\conda\conda\envs\python-CZ4041\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, input_dim=192, activation="relu", kernel_initializer="glorot_normal")`
  


### 2.2.2 MLP Model Training

In [17]:
model_mlp.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01, decay=0.0), metrics = ["accuracy"])

model_mlp.fit(scaled_train, y_cat, epochs=500, validation_split=0.1, batch_size=512)

Train on 891 samples, validate on 99 samples
Epoch 1/500
891/891 [==============================] - 0s 387us/step - loss: 4.7076 - acc: 0.0325 - val_loss: 3.8347 - val_acc: 0.2020
Epoch 2/500
891/891 [==============================] - 0s 13us/step - loss: 3.4594 - acc: 0.3322 - val_loss: 2.8676 - val_acc: 0.5657
Epoch 3/500
891/891 [==============================] - 0s 12us/step - loss: 2.4849 - acc: 0.5836 - val_loss: 1.9711 - val_acc: 0.7273
Epoch 4/500
891/891 [==============================] - 0s 12us/step - loss: 1.7218 - acc: 0.7553 - val_loss: 1.2599 - val_acc: 0.8788
Epoch 5/500
891/891 [==============================] - 0s 12us/step - loss: 1.1159 - acc: 0.8563 - val_loss: 0.8046 - val_acc: 0.9293
Epoch 6/500
891/891 [==============================] - 0s 13us/step - loss: 0.7436 - acc: 0.9237 - val_loss: 0.5064 - val_acc: 0.9798
Epoch 7/500
891/891 [==============================] - 0s 13us/step - loss: 0.4798 - acc: 0.9473 - val_loss: 0.3419 - val_acc: 0.9899
Epoch 8/500
891/

891/891 [==============================] - 0s 11us/step - loss: 0.0035 - acc: 1.0000 - val_loss: 0.0141 - val_acc: 1.0000
Epoch 62/500
891/891 [==============================] - 0s 12us/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 63/500
891/891 [==============================] - 0s 12us/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0138 - val_acc: 1.0000
Epoch 64/500
891/891 [==============================] - 0s 11us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 65/500
891/891 [==============================] - 0s 12us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 66/500
891/891 [==============================] - 0s 12us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0138 - val_acc: 1.0000
Epoch 67/500
891/891 [==============================] - 0s 11us/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0140 - val_acc: 1.0000
Epoch 68/500
891/891 [==============================] - 0s 12us/step

891/891 [==============================] - 0s 12us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0093 - val_acc: 1.0000
Epoch 122/500
891/891 [==============================] - 0s 12us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0093 - val_acc: 1.0000
Epoch 123/500
891/891 [==============================] - 0s 11us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0094 - val_acc: 1.0000
Epoch 124/500
891/891 [==============================] - 0s 12us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0095 - val_acc: 1.0000
Epoch 125/500
891/891 [==============================] - 0s 11us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 126/500
891/891 [==============================] - 0s 12us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0098 - val_acc: 1.0000
Epoch 127/500
891/891 [==============================] - 0s 13us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0104 - val_acc: 1.0000
Epoch 128/500
891/891 [==============================] - 0s 12

891/891 [==============================] - 0s 13us/step - loss: 7.8190e-04 - acc: 1.0000 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 181/500
891/891 [==============================] - 0s 14us/step - loss: 9.5824e-04 - acc: 1.0000 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 182/500
891/891 [==============================] - 0s 12us/step - loss: 8.3926e-04 - acc: 1.0000 - val_loss: 0.0072 - val_acc: 1.0000
Epoch 183/500
891/891 [==============================] - 0s 14us/step - loss: 7.5638e-04 - acc: 1.0000 - val_loss: 0.0071 - val_acc: 1.0000
Epoch 184/500
891/891 [==============================] - 0s 12us/step - loss: 7.0288e-04 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 185/500
891/891 [==============================] - 0s 12us/step - loss: 8.4136e-04 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 186/500
891/891 [==============================] - 0s 11us/step - loss: 7.6154e-04 - acc: 1.0000 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 187/500
891/891 [===========

891/891 [==============================] - 0s 11us/step - loss: 6.3800e-04 - acc: 1.0000 - val_loss: 0.0073 - val_acc: 1.0000
Epoch 239/500
891/891 [==============================] - 0s 12us/step - loss: 4.5981e-04 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 240/500
891/891 [==============================] - 0s 12us/step - loss: 5.1029e-04 - acc: 1.0000 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 241/500
891/891 [==============================] - 0s 11us/step - loss: 5.7653e-04 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 242/500
891/891 [==============================] - 0s 11us/step - loss: 5.2900e-04 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 243/500
891/891 [==============================] - 0s 11us/step - loss: 9.0620e-04 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 1.0000
Epoch 244/500
891/891 [==============================] - 0s 11us/step - loss: 4.7484e-04 - acc: 1.0000 - val_loss: 0.0058 - val_acc: 1.0000
Epoch 245/500
891/891 [===========

891/891 [==============================] - 0s 13us/step - loss: 3.8537e-04 - acc: 1.0000 - val_loss: 0.0053 - val_acc: 1.0000
Epoch 297/500
891/891 [==============================] - 0s 13us/step - loss: 4.5555e-04 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 298/500
891/891 [==============================] - 0s 11us/step - loss: 3.5535e-04 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 299/500
891/891 [==============================] - 0s 11us/step - loss: 3.6148e-04 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 300/500
891/891 [==============================] - 0s 12us/step - loss: 4.1292e-04 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 301/500
891/891 [==============================] - 0s 12us/step - loss: 3.8225e-04 - acc: 1.0000 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 302/500
891/891 [==============================] - 0s 12us/step - loss: 4.8325e-04 - acc: 1.0000 - val_loss: 0.0055 - val_acc: 1.0000
Epoch 303/500
891/891 [===========

891/891 [==============================] - 0s 11us/step - loss: 3.6577e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 355/500
891/891 [==============================] - 0s 11us/step - loss: 3.1463e-04 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 356/500
891/891 [==============================] - 0s 11us/step - loss: 3.5481e-04 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 357/500
891/891 [==============================] - 0s 12us/step - loss: 3.7546e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 358/500
891/891 [==============================] - 0s 11us/step - loss: 3.2978e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 359/500
891/891 [==============================] - 0s 11us/step - loss: 3.3478e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 360/500
891/891 [==============================] - 0s 11us/step - loss: 3.0626e-04 - acc: 1.0000 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 361/500
891/891 [===========

891/891 [==============================] - 0s 11us/step - loss: 2.2558e-04 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 413/500
891/891 [==============================] - 0s 12us/step - loss: 2.4117e-04 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 414/500
891/891 [==============================] - 0s 13us/step - loss: 2.1703e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 415/500
891/891 [==============================] - 0s 13us/step - loss: 2.4168e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 416/500
891/891 [==============================] - 0s 13us/step - loss: 2.5114e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 417/500
891/891 [==============================] - 0s 11us/step - loss: 2.5881e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 418/500
891/891 [==============================] - 0s 11us/step - loss: 2.3099e-04 - acc: 1.0000 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 419/500
891/891 [===========

891/891 [==============================] - 0s 11us/step - loss: 1.8953e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 471/500
891/891 [==============================] - 0s 11us/step - loss: 1.8211e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 472/500
891/891 [==============================] - 0s 11us/step - loss: 2.2631e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 473/500
891/891 [==============================] - 0s 12us/step - loss: 1.8644e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 474/500
891/891 [==============================] - 0s 11us/step - loss: 1.5509e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 475/500
891/891 [==============================] - 0s 11us/step - loss: 1.7093e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 476/500
891/891 [==============================] - 0s 12us/step - loss: 1.7812e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 477/500
891/891 [===========

### 2.2.3 MLP Model Prediction

In [18]:
pred_mlp = model_mlp.predict(scaled_test)

## 2.3 Logistic Regression

### 2.3.1 Logistic Regression Architecture

In [19]:
model_logreg = LogisticRegression(penalty='l2', 
                             dual=False, 
                             tol=0.0001, 
                             C=1.0, 
                             fit_intercept=True, 
                             intercept_scaling=1, 
                             class_weight=None, 
                             random_state=3141593, 
                             solver='lbfgs', 
                             max_iter=100, 
                             multi_class='multinomial', 
                             verbose=1, 
                             warm_start=True, 
                             n_jobs= -1)

### 2.3.2 Grid Search

In [20]:
classifier = GridSearchCV(model_logreg, 
                          param_grid = {'C':[0.1,0.5,1,10, 50, 100, 500, 1000, 2000], 
                                        'tol': [0.001, 0.005,0.0001]}, 
                          scoring='neg_log_loss', 
                          refit='True', 
                          n_jobs=1, 
                          cv=10)

### 2.3.3 Logistic Regression Model Training

In [21]:
classifier.fit(scaled_train, labels)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend Loky

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend Loky

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=-1)]: Using backend Loky

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend Loky

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=-1, penalty='l2', random_state=3141593, solver='lbfgs',
          tol=0.0001, verbose=1, warm_start=True),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'C': [0.1, 0.5, 1, 10, 50, 100, 500, 1000, 2000], 'tol': [0.001, 0.005, 0.0001]},
       pre_dispatch='2*n_jobs', refit='True', return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

### 2.3.4 Logistic Regression Model Prediction

In [22]:
pred_logreg = classifier.predict_proba(scaled_test)

# 3. Ensemble Model

In [23]:
# Save submission file with the weight of each model (i.e. CNN, MLP, LogReg) as the suffix.
def save_submission_file(pred, cnn_weight, mlp_weight, logreg_weight):
    sample_sub = pd.read_csv('sample_submission.csv')
    submission = pd.DataFrame(data = pred)
    submission.columns = sample_sub.columns[1:]
    submission['id'] = sample_sub['id']
    submission.to_csv("ensemble/submission_{}{}{}.csv".format(cnn_weight, mlp_weight, logreg_weight), sep=',', index = False)

In [24]:
for i in range(11):
    for j in range(11):
        for k in range(11):
            if (i + j + k) == 10:
                pred_ensembled = i / 10 * pred_cnn + j / 10 * pred_mlp + k / 10 * pred_logreg
                save_submission_file(pred_ensembled, i, j, k)